In [1]:
!pip install scikit-learn pandas opendatasets matplotlib plotly seaborn numpy --quiet


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import pandas as pd
import opendatasets as od
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import root_mean_squared_error, accuracy_score

%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10,6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'
plt.figure(figsize=(80,20))


<Figure size 8000x2000 with 0 Axes>

<Figure size 8000x2000 with 0 Axes>

In [3]:
# Download latest version
path = od.download("https://www.kaggle.com/c/rossmann-store-sales")

100%|██████████| 6.99M/6.99M [00:00<00:00, 23.3MB/s]



Extracting archive ./rossmann-store-sales/rossmann-store-sales.zip to ./rossmann-store-sales


In [4]:
os.listdir('rossmann-store-sales')

['test.csv', 'train.csv', 'sample_submission.csv', 'store.csv']

In [5]:
ross_df = pd.read_csv('./rossmann-store-sales/train.csv', low_memory=False)

In [6]:
store_df = pd.read_csv('./rossmann-store-sales/store.csv')

In [7]:
merged_df = ross_df.merge(store_df, how='left', on='Store')

In [31]:
test_df = pd.read_csv('./rossmann-store-sales/test.csv')

In [32]:
merged_test_df = test_df.merge(store_df, how='left', on='Store')

In [ ]:
round(merged_df.describe().T,2)

In [ ]:
#check for duplicate
merged_df.duplicated().sum()

In [ ]:
merged_df.isna().sum()

In [ ]:
merged_df.StateHoliday.value_counts()

In [ ]:
sns.histplot(data=merged_df, x='Sales')

In [ ]:
# in this exercise we want to keep the data when they close
merged_df.Open.value_counts()

In [ ]:
# keeping Open = 0 is generating a big 0 predictions
merged_df = merged_df.loc[merged_df['Open'] == 1]
# change Date to timestamp
merged_df['Date'] = pd.to_datetime(merged_df.Date)

In [ ]:
sns.histplot(data=merged_df, x='Sales')

In [ ]:
sns.barplot(data=merged_df, x='DayOfWeek', y='Sales')

In [ ]:
sns.barplot(data=merged_df, x='Promo', y='Sales')

In [ ]:
merged_df.select_dtypes(include=np.number).corr()['Sales'].sort_values(ascending=False)

In [ ]:
scatter_sample_df = merged_df.sample(40000)
sns.scatterplot(x=scatter_sample_df.Sales, y=scatter_sample_df.Customers, hue=scatter_sample_df.Date.dt.year, alpha=0.8)
plt.title('Sales vs Customers')
plt.show()

In [10]:
def prep_train_df(merged_df):
    # feature engineering, split date to day, month, and year
    merged_df['Date'] = pd.to_datetime(merged_df.Date)
    merged_df['Day'] = merged_df.Date.dt.day
    merged_df['Month'] = merged_df.Date.dt.month
    merged_df['Year'] = merged_df.Date.dt.year

    # drop Customers since its not in the test data
    merged_df = merged_df.drop(['Customers'], axis=1)

    # set aside data for validation
    # with this method we get 0.18 error
    train_size = int(.75 * len(merged_df))
    train_df, val_df = merged_df[:train_size], merged_df[train_size:]  

    # with this method we get 0.35 error
    # train_df, val_df = merged_df[merged_df.Date.dt.year <= 2014], merged_df[merged_df.Date.dt.year == 2015]

    # set the targets
    train_targets, val_targets = train_df['Sales'], val_df['Sales']
    
    # drop Sales since its the targets
    train_df, val_df = train_df.drop(['Sales'], axis=1), val_df.drop(['Sales'], axis=1)                 

    # drop column with large missing data
    # n_large_missing_data = [
    #     'Open','Promo2SinceWeek','Promo2SinceYear','CompetitionDistance',
    #     'CompetitionOpenSinceMonth','CompetitionOpenSinceYear'
    # ]
    # c_large_missing_data = ['PromoInterval','Assortment']
    n_large_missing_data = []
    c_large_missing_data = []
    numeric_cols = list(filter(
        lambda x: x not in n_large_missing_data, 
        train_df.select_dtypes(include=np.number).columns.tolist()
    ))
    categorical_cols = list(filter(
        lambda x: x not in c_large_missing_data, 
        train_df.select_dtypes('object').columns.tolist()
    ))

    return train_df, val_df, train_targets, val_targets, numeric_cols, categorical_cols

In [11]:
train_df, val_df, train_targets, val_targets, numeric_cols, categorical_cols = prep_train_df(merged_df)

In [ ]:
sns.barplot(data=merged_df, x='Year', y='Sales')

In [ ]:
sns.barplot(data=merged_df, x='Month', y='Sales')

In [ ]:
sns.barplot(data=merged_df, x='Day', y='Sales')

In [ ]:
merged_df.isna().sum()

In [ ]:
numeric_cols

In [ ]:
categorical_cols

In [ ]:
train_df, val_df, train_targets, val_targets, numeric_cols, categorical_cols = prep_train_df(merged_df)

In [12]:
def enrich_train_df(train_df,val_df):
    # turn categorical features to numeric
    encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
    encoder.fit(train_df[categorical_cols])

    encoded_cols = list(encoder.get_feature_names_out(categorical_cols))

    train_df[encoded_cols] = encoder.transform(train_df[categorical_cols])
    val_df[encoded_cols] = encoder.transform(val_df[categorical_cols])

    train_df = train_df[numeric_cols + encoded_cols]
    val_df = val_df[numeric_cols + encoded_cols]

    # fill missing data with imputation
    imputer = SimpleImputer(strategy = 'mean')
    imputer.fit(train_df)
    train_df = imputer.transform(train_df)
    imputer.fit(val_df)
    val_df = imputer.transform(val_df)
    
    #scale the values
    scaler = MinMaxScaler()  
    scaler.fit(train_df)
    train_df = scaler.transform(train_df)
    scaler.fit(val_df)
    val_df = scaler.transform(val_df)

    return train_df, val_df

In [13]:
train_df, val_df = enrich_train_df(train_df, val_df)

In [ ]:
# setup benchmark with simple models
def return_mean(inputs):
    return np.full(len(inputs), merged_df.Sales.mean())

In [ ]:
train_preds = return_mean(train_df)

In [ ]:
train_preds

In [ ]:
root_mean_squared_error(train_preds, train_targets)

In [ ]:
def guess_random(inputs):
    lo, hi = merged_df.Sales.min(), merged_df.Sales.max()
    return np.random.random(len(inputs)) * (hi - lo) + lo

In [ ]:
train_preds = guess_random(train_df)

In [ ]:
train_preds

In [ ]:
root_mean_squared_error(train_preds, train_targets)

In [ ]:
# baseline ML model
linreg = LinearRegression()

linreg.fit(train_df, train_targets)

In [ ]:
linreg.score(train_df, train_targets)

In [ ]:
train_preds = linreg.predict(train_df)

In [ ]:
train_preds

In [ ]:
root_mean_squared_error(train_preds, train_targets)

In [ ]:
def try_model(model, train_df, train_targets, val_df, val_targets):
    model.fit(train_df, train_targets)

    print(type(model).__name__ + ' train error: {}'.format(1 - model.score(train_df, train_targets)))
    
    train_preds = model.predict(train_df)

    print(type(model).__name__ + ' train - rmse: {}'.format(root_mean_squared_error(train_preds, train_targets)))
    
    print(type(model).__name__ + ' val error: {}'.format(1 - model.score(val_df, val_targets)))

    val_preds = model.predict(val_df)
    
    print(type(model).__name__ + ' val - rmse: {}'.format(root_mean_squared_error(val_preds, val_targets)))

In [ ]:
for mdl in [LinearRegression(), Ridge(), Lasso(), ElasticNet(), SGDRegressor()]:
    try_model(mdl, train_df, train_targets, val_df, val_targets)

In [ ]:
try_model(DecisionTreeRegressor(random_state=42), train_df, train_targets, val_df, val_targets)

In [ ]:
try_model(RandomForestRegressor(random_state=42, n_jobs=-1), train_df, train_targets, val_df, val_targets)
# try_model(RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=20), train_df, train_targets, val_df, val_targets)

In [ ]:
def test_rf_params(param_name, param_value, train_df, train_targets, val_df, val_targets):
    params = {param_name :param_value}
    model = RandomForestRegressor(random_state=42, n_jobs=-1, **params)
    model.fit(train_df, train_targets)
    train_acc = 1 - model.score(train_df, train_targets)
    val_acc = 1 - model.score(val_df, val_targets)
    print('Param Name: {param_name},Param Value: {param_value},Training Error: {train_acc},Validation Error: {val_acc}'.format(
        param_name=param_name, param_value=param_value, train_acc=train_acc, val_acc=val_acc
    ))
    return {'Param Name' : param_name, 'Param Value' : param_value, 'Training Error': train_acc, 'Validation Error': val_acc}

In [ ]:
%%time
# params = [{'name': 'n_estimators','value':val} for val in range(16,60,2)]
params = [{'name': 'n_estimators','value':val} for val in [100]]
errors_df = pd.DataFrame(
    [
        test_rf_params(param['name'], param['value'], train_df, train_targets, val_df, val_targets) 
        for param in params
    ]
)

In [ ]:
errors_df

In [51]:
# single model testing
model = RandomForestRegressor(
    random_state=42, 
    n_jobs=-1,
    n_estimators=76,
    max_depth=22,
    max_features=4,
    max_samples=600020
)

model.fit(train_df, train_targets)

print('train acc: {}'.format(1 - model.score(train_df, train_targets)))
    
train_preds = model.predict(train_df)

print('train - rmse: {}'.format(root_mean_squared_error(train_preds, train_targets)))
    
print('val acc: {}'.format(1 - model.score(val_df, val_targets)))

val_preds = model.predict(val_df)
    
print('val - rmse: {}'.format(root_mean_squared_error(val_preds, val_targets)))

train acc: 0.07182946087154718
train - rmse: 1042.5320261725647
val acc: 0.13022913341589126
val - rmse: 1342.6220145503798


In [33]:
def prep_test_df(merged_test_df):
    # feature engineering, split date to day, month, and year
    submit_test_df = merged_test_df.copy()
    submit_test_df['Date'] = pd.to_datetime(submit_test_df.Date)
    submit_test_df['Day'] = submit_test_df.Date.dt.day
    submit_test_df['Month'] = submit_test_df.Date.dt.month
    submit_test_df['Year'] = submit_test_df.Date.dt.year

    # drop column with large missing data
    # n_large_missing_data = [
    #     'Open','Promo2SinceWeek','Promo2SinceYear','CompetitionDistance',
    #     'CompetitionOpenSinceMonth','CompetitionOpenSinceYear'
    # ]
    # c_large_missing_data = ['PromoInterval','Assortment']
    n_large_missing_data = ['Id']
    c_large_missing_data = []
    numeric_cols = list(filter(
        lambda x: x not in n_large_missing_data, 
        submit_test_df.select_dtypes(include=np.number).columns.tolist()
    ))
    categorical_cols = list(filter(
        lambda x: x not in c_large_missing_data, 
        submit_test_df.select_dtypes('object').columns.tolist()
    ))
    
    return submit_test_df, numeric_cols, categorical_cols

In [34]:
submit_test_df, numeric_cols, categorical_cols = prep_test_df(merged_test_df)

In [ ]:
numeric_cols

In [ ]:
categorical_cols

In [ ]:
merged_test_df.isna().sum()

In [ ]:
# test has only 0 and a, so just edit the file to change the first couple with b and c
submit_test_df.StateHoliday.value_counts()

In [ ]:
submit_test_df.isna().sum()

In [35]:
def enrich_test_df(submit_test_df):
    # turn categorical features to numeric
    encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
    encoder.fit(submit_test_df[categorical_cols])

    encoded_cols = list(encoder.get_feature_names_out(categorical_cols))

    submit_test_df[encoded_cols] = encoder.transform(submit_test_df[categorical_cols])
    
    submit_test_df = submit_test_df[numeric_cols + encoded_cols]

    # fill missing data with imputation
    imputer = SimpleImputer(strategy = 'mean')
    imputer.fit(submit_test_df)
    submit_test_df = imputer.transform(submit_test_df)
    
    #scale the values
    scaler = MinMaxScaler()  
    scaler.fit(submit_test_df)
    submit_test_df = scaler.transform(submit_test_df)

    return submit_test_df

In [36]:
X_inputs = enrich_test_df(submit_test_df)

In [37]:
test_preds = model.predict(X_inputs)
test_preds

array([ 6215.69254539,  7058.59164975, 11983.39415803, ...,
        5492.33162157, 18784.05591063,  5905.16567254])

In [38]:
test_preds_df = pd.DataFrame(test_preds, columns=['Sales'])

In [39]:
test_preds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Sales   41088 non-null  float64
dtypes: float64(1)
memory usage: 321.1 KB


In [40]:
merged_test_df['Sales'] = test_preds_df['Sales']

In [41]:
merged_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Id                         41088 non-null  int64  
 1   Store                      41088 non-null  int64  
 2   DayOfWeek                  41088 non-null  int64  
 3   Date                       41088 non-null  object 
 4   Open                       41077 non-null  float64
 5   Promo                      41088 non-null  int64  
 6   StateHoliday               41088 non-null  object 
 7   SchoolHoliday              41088 non-null  int64  
 8   StoreType                  41088 non-null  object 
 9   Assortment                 41088 non-null  object 
 10  CompetitionDistance        40992 non-null  float64
 11  CompetitionOpenSinceMonth  25872 non-null  float64
 12  CompetitionOpenSinceYear   25872 non-null  float64
 13  Promo2                     41088 non-null  int

In [42]:
merged_test_df.Sales.isna().sum()

np.int64(0)

In [43]:
submission_df  = pd.read_csv('./rossmann-store-sales/sample_submission.csv')

In [44]:
submission_df['Sales'] = merged_test_df['Open'].fillna(1).astype('int') * test_preds

In [45]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      41088 non-null  int64  
 1   Sales   41088 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 642.1 KB


In [46]:
submission_df.Id.isna().sum()

np.int64(0)

In [47]:
submission_df.Sales.isna().sum()

np.int64(0)

In [48]:
submission_df.Sales.value_counts()

Sales
0.000000        5984
5752.128047        2
6435.460581        2
6134.553791        2
4320.512224        1
                ... 
10688.385202       1
11179.136211       1
9616.283850        1
8075.648599        1
9809.419919        1
Name: count, Length: 35102, dtype: int64

In [49]:
submission_df.to_csv('submission.csv', index=None)

In [50]:
!head submission.csv

Id,Sales
1,6215.692545393203
2,7058.591649752853
3,11983.394158028179
4,8954.46857321445
5,10064.772361487443
6,8300.930697283979
7,15920.61362179487
8,11169.767753771512
9,8401.047211648953
